In [ ]:
pip install scikeras

In [1]:
from numpy.random import seed
seed(1)
import tensorflow 
tensorflow.random.set_seed(1) 
import pandas as pd
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from collections import Counter
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D, Convolution1D
from keras.optimizers import Adadelta
from sklearn import metrics
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import json
import time

2025-05-26 23:36:00.826669: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-26 23:36:00.826689: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to /home/alice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def remove_stopwords(text):
    all_words = text.split(" ")
    clean_text = [i for i in all_words if i not in stopwords and i!=""]
    return " ".join(clean_text)

In [3]:
def remove_html_tags(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [4]:
def lower_texts(text):
    return text.lower()

In [5]:
def clean_text(text):
    clean_text = remove_stopwords(text)
    clean_text = remove_html_tags(clean_text)
    clean_text = lower_texts(clean_text)
    return clean_text

In [6]:
def create_model_age(filters = [100], kernel_size = [50], strides = [100], 
                 dropout_rate = 0.5, pool_size = [5], dense_units = 100, max_len = 1000):

    model = Sequential()

    # conv 1
    model.add(Conv1D(filters = filters[0], 
                     kernel_size = kernel_size[0],
                     strides = strides[0], 
                     activation = 'relu', 
                     input_shape = (max_len, 1) ))

    # pooling layer 1
    for i in range(len(pool_size)):
        model.add(MaxPooling1D(pool_size = pool_size[i], strides = 1))
        model.add(Activation('relu'))
    
    #model.add(Activation('relu'))
    
    model.add(Flatten())
    
    if dropout_rate is not None:
        model.add(Dropout(dropout_rate))
        
    model.add(Dense(units = dense_units, activation = 'relu'))
    model.add(Dense(units = 3, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer = Adadelta(
       learning_rate=1, name="Adadelta"
    ), metrics = ['accuracy'])
    return model

In [7]:
import datetime

def formatTime(seg):
    return str(datetime.timedelta(seconds=seg))

In [9]:
df = pd.read_csv("../data/blogset-without-duplicate.csv")

y = df["Age"]

df_train, df_test = train_test_split(df, test_size=0.2, random_state=115, stratify=y)

# df_train = pd.read_csv('../data/train_1.csv')
# df_test = pd.read_csv('../data/test_1.csv')

X_train = df_train["Texts"].apply(clean_text).to_numpy()
X_test = df_test["Texts"].apply(clean_text).to_numpy()
y_train = df_train["Age"].to_numpy()
y_test = df_test["Age"].to_numpy()

num_words = []
for t in X_train:
    num_words.append(len(t.split()))

mean = sum(num_words)//len(num_words)

train_texts = X_train.tolist()
test_texts = X_test.tolist()
tfidfvec = TfidfVectorizer(max_features = mean, max_df=0.9)
tfidfvec.fit(train_texts)
tfidf_train = tfidfvec.transform(train_texts).toarray()
tfidf_test = tfidfvec.transform(test_texts).toarray()

X_train = tfidf_train.reshape(tfidf_train.shape[0],tfidf_train.shape[1],1)
X_test = tfidf_test.reshape(tfidf_test.shape[0],tfidf_test.shape[1],1)
y_train = keras.utils.to_categorical(y_train,num_classes=3)
y_test = keras.utils.to_categorical(y_test,num_classes=3)

size = X_test.shape[1]

In [11]:
from keras.callbacks import EarlyStopping 

callback = [
        EarlyStopping(patience=5, monitor='val_accuracy', mode='max', restore_best_weights=True),
    ]

In [12]:
size

3769

In [13]:
def create_model_age1(filters = [100], kernel_size = [50], strides = [100], 
                 dropout_rate = 0.5, pool_size = [5], dense_units = 100, max_len = 1000):

    model = Sequential()

    # conv 1
    model.add(Conv1D(filters = filters[0], 
                     kernel_size = kernel_size[0],
                     strides = strides[0], 
                     activation = 'relu', 
                     input_shape = (max_len, 1) ))

    # pooling layer 1
    for i in range(len(pool_size)):
        model.add(MaxPooling1D(pool_size = pool_size[i], strides = 1))
        model.add(Activation('relu'))
    
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    if dropout_rate is not None:
        model.add(Dropout(dropout_rate))
        
    model.add(Dense(units = dense_units, activation = 'relu'))
    model.add(Dense(units = 3, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer = Adadelta(
       learning_rate=1, name="Adadelta"
    ), metrics = ['accuracy'])
    return model

In [14]:
X_test.shape

(508, 3769, 1)

In [15]:
for i in range(1):
    X_train = tfidf_train.reshape(tfidf_train.shape[0],tfidf_train.shape[1],1)
    X_test = tfidf_test.reshape(tfidf_test.shape[0],tfidf_test.shape[1],1)

    y_train_age = df_train["Age"].to_numpy()
    y_test_age = df_test["Age"].to_numpy()

    y_train_age = keras.utils.to_categorical(y_train_age,num_classes=3)
    y_test_age_cat = keras.utils.to_categorical(y_test_age,num_classes=3)

    model_age = create_model_age1([60], 
                                        kernel_size=[1], 
                                        strides=[1], 
                                        dropout_rate=0.6, 
                                        pool_size=[4], 
                                        dense_units=512, 
                                        max_len=X_test.shape[1])

    model_age.fit(X_train,y_train_age,validation_data=(X_test,y_test_age_cat), 
                    batch_size=32,
                    epochs=10,
                    callbacks=callback)

    y_pred_age = model_age.predict(
        X_test
    )

    y_pred_list_age = [np.argmax(x, axis=-1) for x in y_pred_age]

    print(metrics.f1_score(y_test_age, y_pred_list_age, average='macro'))

2025-05-26 23:39:03.332244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-05-26 23:39:03.333000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-26 23:39:03.333455: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-05-26 23:39:03.333518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-05-26 23:39:03.333728: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Epoch 1/10


2025-05-26 23:39:04.073273: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 462766080 exceeds 10% of free system memory.
2025-05-26 23:39:04.100538: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 462766080 exceeds 10% of free system memory.


64/64 [==============================] - 35s 532ms/step - loss: 1.0615 - accuracy: 0.4293 - val_loss: 1.0262 - val_accuracy: 0.4724
Epoch 2/10
64/64 [==============================] - 33s 520ms/step - loss: 0.9792 - accuracy: 0.5057 - val_loss: 0.9891 - val_accuracy: 0.4744
Epoch 3/10
64/64 [==============================] - 33s 516ms/step - loss: 0.8982 - accuracy: 0.5692 - val_loss: 1.0031 - val_accuracy: 0.4843
Epoch 4/10
64/64 [==============================] - 32s 503ms/step - loss: 0.8127 - accuracy: 0.6259 - val_loss: 1.1395 - val_accuracy: 0.4528
Epoch 5/10
64/64 [==============================] - 32s 503ms/step - loss: 0.7532 - accuracy: 0.6506 - val_loss: 1.2890 - val_accuracy: 0.4370
Epoch 6/10
64/64 [==============================] - 32s 502ms/step - loss: 0.6762 - accuracy: 0.7013 - val_loss: 1.2491 - val_accuracy: 0.4488
Epoch 7/10
64/64 [==============================] - 32s 505ms/step - loss: 0.6134 - accuracy: 0.7260 - val_loss: 1.2601 - val_accuracy: 0.4508
Epoch 8/10

NameError: name 'np' is not defined

In [30]:
model_age.save('final_model.keras')

In [16]:
import numpy as np


y_pred_list_age = [np.argmax(x, axis=-1) for x in y_pred_age]

print(metrics.f1_score(y_test_age, y_pred_list_age, average='macro'))

0.48256185382192296


In [17]:
df_test['Predict']=y_pred_list_age

In [18]:
metrics.f1_score(y_test_age, y_pred_list_age, average='macro')

0.48256185382192296

In [25]:
dict_class_2 = pd.read_csv('../data/dict_class_2.csv', encoding='latin-1')

df_test = df_test.reset_index(drop=True)
dict_class_2 = dict_class_2.reset_index(drop=True)

df_test['AuthorID'] = df_test['AuthorID'].astype(int)
dict_class_2['AuthorID'] = dict_class_2['AuthorID'].astype(int)

compare_test_2 = df_test[df_test['AuthorID'].isin(dict_class_2['AuthorID'])]

compare_test_2 = compare_test_2[['AuthorID', 'Age', 'Texts', f'Predict']]
compare_dict_2 = dict_class_2[['AuthorID', 'c']]

compare_2 = compare_test_2.merge(compare_dict_2, left_on='AuthorID', right_on='AuthorID')

print(f"DICT F1:{metrics.f1_score(compare_2['Age'], compare_2['c'], average='macro')})")
print(f"TRAD F1: {metrics.f1_score(compare_2['Age'], compare_2[f'Predict'], average='macro')}")
#compare_2.shape, metrics.f1_score(compare_2['Age'], compare_2[f'Predict'], average='macro'), metrics.f1_score(compare_2['Age'], compare_2['c'], average='macro')

DICT F1:0.49968992248062016)
TRAD F1: 0.5167515341428385


In [27]:
print(f"DIC ACC:{metrics.accuracy_score(compare_2['Age'], compare_2['c'])})")
print(f"DICT ACC: {metrics.accuracy_score(compare_2['Age'], compare_2[f'Predict'])}")

DIC ACC:0.5421686746987951)
DICT ACC: 0.5180722891566265


In [28]:
merged_df_2 = df_test.merge(compare_2[['AuthorID', 'c']], on='AuthorID', how='left')
merged_df_2[f'Predict'] = merged_df_2['c'].combine_first(merged_df_2[f'Predict'])

print(f"DICT F1:{metrics.f1_score(merged_df_2['Age'], merged_df_2[f'Predict'], average='macro')})")
print(f"TRAD F1: {metrics.f1_score(df_test['Age'], df_test[f'Predict'], average='macro')}")

DICT F1:0.49086251439192624)
TRAD F1: 0.48256185382192296


In [29]:
print(f"DICT F1:{metrics.accuracy_score(merged_df_2['Age'], merged_df_2[f'Predict'])})")
print(f"TRAD F1: {metrics.accuracy_score(df_test['Age'], df_test[f'Predict'])}")

DICT F1:0.4881889763779528)
TRAD F1: 0.484251968503937


In [31]:
compare_2.shape[1]/df_test.shape[1]

0.8333333333333334